In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import isnan
from pandas import read_csv
from datetime import datetime

In [49]:
client = pd.read_csv('client.csv')
account = pd.read_csv('account.csv')
card = pd.read_csv('card.csv')
disp = pd.read_csv('disp.csv')
district = pd.read_csv('district.csv')
loan = pd.read_csv('loan.csv')
order = pd.read_csv('order.csv')
trans = pd.read_csv('trans.csv')

C:\Users\HD\AppData\Local\Temp\ipykernel_3616\228495804.py:8: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  trans = pd.read_csv('trans.csv')


## Data Understanding : Client Dataset

In [3]:
print("Client columns:", client.columns)

Client columns: Index(['client_id', 'birth_number', 'district_id'], dtype='object')


client_id : client identifier 

birth_number : birthday and sex

district_id : address of the client

In [4]:
client.head()

,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1
3,4,561201,5
4,5,605703,5


In [5]:
client.shape

(5369, 3)

In [6]:
# Vérifier l'unicité des client_id
duplicate_client_ids = client[client.duplicated(subset='client_id', keep=False)]

if duplicate_client_ids.empty:
    print("Tous les client_id sont uniques.")
else:
    print("Il y a des duplications de client_id:")
    print(duplicate_client_ids)

Tous les client_id sont uniques.


In [7]:
client.isnull().values.any()

False

The column birth_number representes birthday and sex where : 
The value is in the form: YYMMDD (for men) & YYMM+50DD (for women)

So we will extract from birth_number the columns : birth_date, age and sex.

In [51]:
from datetime import datetime

def extract_info(birth_number):
    birth_number_str = str(birth_number).zfill(6)
    year = int(birth_number_str[:2])
    month = int(birth_number_str[2:4])
    day = int(birth_number_str[4:6])

    if month > 50:
        sex = 'Female'
        month -= 50
    else:
        sex = 'Male'

    if year <= int(datetime.now().strftime('%y')):
        year += 2000
    else:
        year += 1900

    birth_date = datetime(year, month, day)
    reference_date = datetime(1999, 1, 1)
    age = (reference_date  - birth_date).days // 365

    return birth_date, age, sex

# Appliquer la fonction à la colonne birth_number
client[['birth_date', 'age', 'sex']] = client['birth_number'].apply(lambda x: pd.Series(extract_info(x)))

# Supprimer la colonne birth_number
client.drop(columns=['birth_number'], inplace=True)

# Afficher le dataset avec les nouvelles colonnes
print(client)

      client_id  district_id birth_date  age     sex
0             1           18 1970-12-13   28  Female
1             2            1 1945-02-04   53    Male
2             3            1 1940-10-09   58  Female
3             4            5 1956-12-01   42    Male
4             5            5 1960-07-03   38  Female
...         ...          ...        ...  ...     ...
5364      13955            1 1945-10-30   53  Female
5365      13956            1 1943-04-06   55    Male
5366      13968           61 1968-04-13   30    Male
5367      13971           67 1962-10-19   36  Female
5368      13998           74 1953-08-12   45  Female

[5369 rows x 5 columns]


## Data Understanding : Account Dataset

In [9]:
print("Account columns:", account.columns)

Account columns: Index(['account_id', 'district_id', 'frequency', 'date'], dtype='object')


account_id : Identification of the account

district_id : Location of the branch

frequency : Frequency of statement issuance	

date : Date of the account's creation

In [10]:
account.shape

(4500, 4)

In [11]:
# Vérifier l'unicité des account_id
duplicate_account_ids = account[account.duplicated(subset='account_id', keep=False)]

if duplicate_account_ids.empty:
    print("Tous les account_id sont uniques.")
else:
    print("Il y a des duplications de account_id:")
    print(duplicate_account_ids)

Tous les account_id sont uniques.


In [12]:
account.isnull().values.any()

False

In [13]:
account.head()

,account_id,district_id,frequency,date
0,576,55,POPLATEK MESICNE,930101
1,3818,74,POPLATEK MESICNE,930101
2,704,55,POPLATEK MESICNE,930101
3,2378,16,POPLATEK MESICNE,930101
4,2632,24,POPLATEK MESICNE,930102


The date is written in the form YYMMDD, let's change it to datetime form.

In [14]:
from datetime import datetime

def convert_date(date_str):
    date_str = str(date_str).zfill(6)
    year = int(date_str[:2])
    month = int(date_str[2:4])
    day = int(date_str[4:6])
    
    # Assume all years are in the 20th or 21st century
    current_year = datetime.now().year
    if year <= int(str(current_year)[-2:]):  # If year is less than or equal to current year's last two digits
        year += 2000
    else:
        year += 1900
    
    return datetime(year, month, day)

# Convertir la colonne date
account['date'] = account['date'].apply(convert_date)

# Afficher le dataset avec les dates converties
print(account)

      account_id  district_id         frequency       date
0            576           55  POPLATEK MESICNE 1993-01-01
1           3818           74  POPLATEK MESICNE 1993-01-01
2            704           55  POPLATEK MESICNE 1993-01-01
3           2378           16  POPLATEK MESICNE 1993-01-01
4           2632           24  POPLATEK MESICNE 1993-01-02
...          ...          ...               ...        ...
4495         124           55  POPLATEK MESICNE 1997-12-28
4496        3958           59  POPLATEK MESICNE 1997-12-28
4497         777           30  POPLATEK MESICNE 1997-12-28
4498        1573           63  POPLATEK MESICNE 1997-12-29
4499        3276            1  POPLATEK MESICNE 1997-12-29

[4500 rows x 4 columns]


The column frequency is written in Czech, let's translate it : 

"POPLATEK MESICNE" - Monthly Fee

"POPLATEK TYDNE" - Weekly Fee

"POPLATEK PO OBRATU" - Fee After Transaction

In [15]:
# Dictionnaire de traduction pour la colonne frequency
translation_dict = {
    "POPLATEK MESICNE": "Monthly Fee",
    "POPLATEK TYDNE": "Weekly Fee",
    "POPLATEK PO OBRATU": "Fee After Transaction"
}

# Traduire la colonne frequency
account['frequency'] = account['frequency'].map(translation_dict)

# Afficher le dataset avec la colonne frequency traduite
print(account)

      account_id  district_id    frequency       date
0            576           55  Monthly Fee 1993-01-01
1           3818           74  Monthly Fee 1993-01-01
2            704           55  Monthly Fee 1993-01-01
3           2378           16  Monthly Fee 1993-01-01
4           2632           24  Monthly Fee 1993-01-02
...          ...          ...          ...        ...
4495         124           55  Monthly Fee 1997-12-28
4496        3958           59  Monthly Fee 1997-12-28
4497         777           30  Monthly Fee 1997-12-28
4498        1573           63  Monthly Fee 1997-12-29
4499        3276            1  Monthly Fee 1997-12-29

[4500 rows x 4 columns]


In [16]:
# Compter le nombre d'occurrences de chaque type de frais
frequency_counts = account['frequency'].value_counts()

# Afficher les résultats
print(frequency_counts)

frequency
Monthly Fee              4167
Weekly Fee                240
Fee After Transaction      93
Name: count, dtype: int64


## Data Understanding : Card Dataset

In [17]:
print("Card columns:", card.columns)

Card columns: Index(['card_id', 'disp_id', 'type', 'issued'], dtype='object')


card_id : Card Identifier	

disp_id : Disposition to an account	

type : Type of card	('Junior', 'Classic' or 'Gold')

issued : Date card was issued

In [18]:
card.shape

(892, 4)

In [19]:
# Vérifier l'unicité des card_id
duplicate_card_ids = card[card.duplicated(subset='card_id', keep=False)]

if duplicate_card_ids.empty:
    print("Tous les card_id sont uniques.")
else:
    print("Il y a des duplications de card_id:")
    print(duplicate_card_ids)

Tous les card_id sont uniques.


In [20]:
card.head()

,card_id,disp_id,type,issued
0,1005,9285,classic,931107 00:00:00
1,104,588,classic,940119 00:00:00
2,747,4915,classic,940205 00:00:00
3,70,439,classic,940208 00:00:00
4,577,3687,classic,940215 00:00:00


The column issued represents the date the card was issued and it's written in the form YYMMDD, let's change it to datetime.

In [21]:
def convert_date(date_str):
    date_str = str(date_str).zfill(6)
    year = int(date_str[:2])
    month = int(date_str[2:4])
    day = int(date_str[4:6])
    
    # Assume all years are in the 20th or 21st century
    current_year = datetime.now().year
    if year <= int(str(current_year)[-2:]):  # If year is less than or equal to current year's last two digits
        year += 2000
    else:
        year += 1900
    
    return datetime(year, month, day)

# Convertir la colonne issued
card['issued'] = card['issued'].apply(convert_date)

# Afficher le dataset avec les dates converties
print(card)

     card_id  disp_id     type     issued
0       1005     9285  classic 1993-11-07
1        104      588  classic 1994-01-19
2        747     4915  classic 1994-02-05
3         70      439  classic 1994-02-08
4        577     3687  classic 1994-02-15
..       ...      ...      ...        ...
887      125      694     gold 1998-12-26
888      674     4360  classic 1998-12-28
889      322     2063  classic 1998-12-28
890      685     4467  classic 1998-12-29
891      635     4083  classic 1998-12-29

[892 rows x 4 columns]


In [22]:
# Compter le nombre d'occurrences de chaque type de carte
types = card['type'].value_counts()

# Afficher les résultats
print(types)

type
classic    659
junior     145
gold        88
Name: count, dtype: int64


## Data Understanding : Disposition Dataset

In [23]:
print("Disp columns:", disp.columns)

Disp columns: Index(['disp_id', 'client_id', 'account_id', 'type'], dtype='object')


disp_id	: Record Identifier

client_id :	Client Identifier	

account_id : Account Identifier	 

type : Type of Disposition : Owner or User	(Only owner can issue permanent orders and ask for a loan)

In [24]:
# Vérifier l'unicité des disp_id
duplicate_disp_ids = disp[disp.duplicated(subset='disp_id', keep=False)]

if duplicate_disp_ids.empty:
    print("Tous les disp_id sont uniques.")
else:
    print("Il y a des duplications de disp_id:")
    print(duplicate_disp_ids)

Tous les disp_id sont uniques.


In [25]:
# Compter le nombre d'occurrences de chaque type de disposition
disp_types = disp['type'].value_counts()

# Afficher les résultats
print(disp_types)

type
OWNER        4500
DISPONENT     869
Name: count, dtype: int64


## Data Understanding : District Dataset

In [26]:
print("District columns:", district.columns)

District columns: Index(['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11',
       'a12', 'a13', 'a14', 'a15', 'a16'],
      dtype='object')


A1  : District Identifier	 

A2	: District Name	 

A3	: Region	 

A4	: No. of Inhabitants	 

A5	: No. of Municipalities with inhabitants < 499	 

A6	: No. of Municipalities with inhabitants 500-1999	 

A7	: No. of Municipalities with inhabitants 2000-9999	

A8	: No. of Municipalities with inhabitants > 10000	 

A9	: No. of Cities	 

A10	: Ratio of urban inhabitants	 

A11	: Average Salary	 

A12	: Unemployment rate in 1995

A13	: Unemployment rate in 1996	

A14	: No. of Enterpreneurs per 1000 inhabitants	

A15	: No. of Crimes commited in 1995	 

A16	: No. of Crimes commited in 1996

In [27]:
# Vérifier l'unicité des district_id
duplicate_district_ids = district[district.duplicated(subset='a1', keep=False)]

if duplicate_district_ids.empty:
    print("Tous les a1 sont uniques.")
else:
    print("Il y a des duplications de a1:")
    print(duplicate_district_ids)

Tous les a1 sont uniques.


In [28]:
# Dictionnaire de mappage pour les nouveaux noms de colonnes
column_mapping = {
    'a1': 'district_id',
    'a2': 'district_name',
    'a3': 'region',
    'a4': 'num_inhabitants',
    'a5': 'num_municipalities_lt_499',
    'a6': 'num_municipalities_500_1999',
    'a7': 'num_municipalities_2000_9999',
    'a8': 'num_municipalities_gt_10000',
    'a9': 'num_cities',
    'a10': 'urban_inhabitants_ratio',
    'a11': 'average_salary',
    'a12': 'unemployment_rate_1995',
    'a13': 'unemployment_rate_1996',
    'a14': 'entrepreneurs_per_1000',
    'a15': 'crimes_committed_1995',
    'a16': 'crimes_committed_1996'
}

# Renommer les colonnes
district.rename(columns=column_mapping, inplace=True)

# Afficher le dataset avec les nouvelles colonnes
print(district)

    district_id    district_name           region  num_inhabitants  \
0             1      Hl.m. Praha           Prague          1204953   
1             2          Benesov  central Bohemia            88884   
2             3           Beroun  central Bohemia            75232   
3             4           Kladno  central Bohemia           149893   
4             5            Kolin  central Bohemia            95616   
..          ...              ...              ...              ...   
72           73            Opava    north Moravia           182027   
73           74  Ostrava - mesto    north Moravia           323870   
74           75           Prerov    north Moravia           138032   
75           76          Sumperk    north Moravia           127369   
76           77           Vsetin    north Moravia           148545   

    num_municipalities_lt_499  num_municipalities_500_1999  \
0                           0                            0   
1                          80      

## Data Understanding : Loan Dataset

In [29]:
print("Loan columns:", loan.columns)

Loan columns: Index(['loan_id', 'account_id', 'date', 'amount', 'duration', 'payments',
       'status'],
      dtype='object')


loan_id : Loan Identification

account_id : Account Identification 

date : Date in form YYMMDD

amount : Amount of loan

duration : Duration of loan

payments : Monthly Payments on Loan

status : Status in paying off the loan 

    'A' stands for contract finished, no problems
    
    'B' stands for contract finished, loan not payed

    'C' stands for running contract, OK thus-far

    'D' stands for running contract, client in debt

In [30]:
loan.shape

(682, 7)

In [41]:
loan.isnull().values.any()

False

In [31]:
# Vérifier l'unicité des loan_id
duplicate_loan_ids = loan[loan.duplicated(subset='loan_id', keep=False)]

if duplicate_loan_ids.empty:
    print("Tous les loan_id sont uniques.")
else:
    print("Il y a des duplications de loan_id:")
    print(duplicate_loan_ids)

Tous les loan_id sont uniques.


In [32]:
loan.head()

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033.0,B
1,5316,1801,930711,165960,36,4610.0,A
2,6863,9188,930728,127080,60,2118.0,A
3,5325,1843,930803,105804,36,2939.0,A
4,7240,11013,930906,274740,60,4579.0,A


The column dare is written in the form YYMMDD, let's change it to datetime.

In [33]:
def convert_date(date_str):
    date_str = str(date_str).zfill(6)
    year = int(date_str[:2])
    month = int(date_str[2:4])
    day = int(date_str[4:6])
    
    # Assume all years are in the 20th or 21st century
    current_year = datetime.now().year
    if year <= int(str(current_year)[-2:]):  # If year is less than or equal to current year's last two digits
        year += 2000
    else:
        year += 1900
    
    return datetime(year, month, day)

# Convertir la colonne date
loan['date'] = loan['date'].apply(convert_date)

# Afficher le dataset avec les dates converties
print(loan)

     loan_id  account_id       date  amount  duration  payments status
0       5314        1787 1993-07-05   96396        12    8033.0      B
1       5316        1801 1993-07-11  165960        36    4610.0      A
2       6863        9188 1993-07-28  127080        60    2118.0      A
3       5325        1843 1993-08-03  105804        36    2939.0      A
4       7240       11013 1993-09-06  274740        60    4579.0      A
..       ...         ...        ...     ...       ...       ...    ...
677     4989         105 1998-12-05  352704        48    7348.0      C
678     5221        1284 1998-12-05   52512        12    4376.0      C
679     6402        6922 1998-12-06  139488        24    5812.0      C
680     5346        1928 1998-12-06   55632        24    2318.0      C
681     6748        8645 1998-12-08  240900        60    4015.0      C

[682 rows x 7 columns]


In [34]:
# Compter le nombre d'occurrences de chaque status
status_occ = loan['status'].value_counts()

# Afficher les résultats
print(status_occ)

status
C    403
A    203
D     45
B     31
Name: count, dtype: int64


## Data Understanding : Order Dataset

In [35]:
print("Order columns:", order.columns)

Order columns: Index(['order_id', 'account_id', 'bank_to', 'account_to', 'amount',
       'k_symbol'],
      dtype='object')


order_id : Order Identifier

account_id : Account Identifier	 

bank_to : Bank of the recipient (each bank has a two letter code)

account_to : Account of the recipient

amount : Amount debited from order account	

k_symbol : Characterization of the payment

In [38]:
order.shape

(6471, 6)

In [39]:
# Vérifier l'unicité des order_id
duplicate_order_ids = order[order.duplicated(subset='order_id', keep=False)]

if duplicate_order_ids.empty:
    print("Tous les order_id sont uniques.")
else:
    print("Il y a des duplications de order_id:")
    print(duplicate_order_ids)

Tous les order_id sont uniques.


In [40]:
order.isnull().values.any()

False

In [37]:
order.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,


The column k_symbol is written in Czech, let's translate it : 

'POJISTNE' stands for Insurance Payment

'SIPO' stands for Household Payment

'LEASING' stands for Leasing Payment

'UVER' stands for Loan Payment

In [42]:
# Traduction pour la colonne k_symbol
translation_dict = {
    "POJISTNE": "Insurance",
    "SIPO": "Household",
    "LEASING": "Leasing", 
    "UVER": "Loan"
}

# Traduire la colonne k_symbol
order['k_symbol'] = order['k_symbol'].map(translation_dict)

# Afficher le dataset avec la colonne k_symbol traduite
print(order)

      order_id  account_id bank_to  account_to  amount   k_symbol
0        29401           1      YZ    87144583  2452.0  Household
1        29402           2      ST    89597016  3372.7       Loan
2        29403           2      QR    13943797  7266.0  Household
3        29404           3      WX    83084338  1135.0  Household
4        29405           3      CD    24485939   327.0        NaN
...        ...         ...     ...         ...     ...        ...
6466     46334       11362      YZ    70641225  4780.0  Household
6467     46335       11362      MN    78507822    56.0        NaN
6468     46336       11362      ST    40799850   330.0  Insurance
6469     46337       11362      KL    20009470   129.0        NaN
6470     46338       11362      MN    61540514  5392.0       Loan

[6471 rows x 6 columns]


In [43]:
order.isnull().values.any()

True

In [45]:
# Compter le nombre d'occurrences de chaque k_symbol
k_symbol_occ = order['k_symbol'].value_counts(dropna=False)

# Afficher les résultats
print(k_symbol_occ)

k_symbol
Household    3502
NaN          1379
Loan          717
Insurance     532
Leasing       341
Name: count, dtype: int64


In [46]:
# Remplacer les valeurs NaN par "Other"
order['k_symbol'].fillna('Other', inplace=True)

C:\Users\HD\AppData\Local\Temp\ipykernel_3616\652929224.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  order['k_symbol'].fillna('Other', inplace=True)


In [47]:
# Compter le nombre d'occurrences de chaque k_symbol
k_symbol_occ = order['k_symbol'].value_counts(dropna=False)

# Afficher les résultats
print(k_symbol_occ)

k_symbol
Household    3502
Other        1379
Loan          717
Insurance     532
Leasing       341
Name: count, dtype: int64


## Data Understanding : Transactions Dataset